In [ ]:
from qdrant_client import QdrantClient
import os
from qdrant_client.http.models import VectorParams, Distance
from semantic_text_splitter import TextSplitter

In [ ]:
EMBEDDING_API_URL = "http://ws-04.wade0426.me/embed"

In [ ]:
client = client = QdrantClient(url="http://localhost:6333")

In [ ]:
try:
    client.create_collection(
        collection_name = "data_01_collection",
        vectors_config = VectorParams(size=4096, distance=Distance.COSINE)
    )
except Exception as e:
    print(f"* failed {e}")

In [ ]:
text_list = ["data_01.txt", "data_02.txt", "data_03.txt", "data_04.txt", "data_05.txt"]
text_path = "./CW"
documents_data = []

In [ ]:
for file_name in text_list:
    combined_path = os.path.join(text_path, file_name)
    try:
        with open(combined_path, "r", encoding="utf-8") as file:
            content = file.read()
            if content:
                documents_data.append({
                    "file_name": file_name,
                    "text": content
                })
    except FileNotFoundError:
        print(f"* not found {combined_path}")


In [ ]:
def get_embedding(texts):
    # 1. 確保輸入是 list
    if isinstance(texts, str):
        texts = [texts]

    payload = {
        "texts": texts,
        "normalize": True,
        "batch_size": 32
    }

    try:
        response = requests.post(EMBEDDING_API_URL, json=payload, timeout=10)

        # ✅ 修正點：是 raise (拋出)，不是 rise
        response.raise_for_status()

        result = response.json()

        # ✅ 建議改用 .get()，防止 API 回傳的 key 不叫 "embedding" 時程式崩潰
        # 通常這類 API 的 key 可能是 "embeddings" (複數) 或 "data"，請確認你的 API 文件
        return result.get("embedding", [])

    except Exception as e:
        print(f"* Embedding failed: {e}")
        return []

In [ ]:
 with open(combined_path, "r", encoding="utf-8") as f:
     content = f.read()
     vectors = get_embedding(content)

In [ ]:
def semantic_splitter(text):
    splitter = TextSplitter(200, 1000)
    return splitter.chunks(text)

In [ ]:
import requests

API_URL = "http://ws-04.wade0426.me/embed"

def check_dimension():
    payload = {"texts": ["測試維度"]} # 使用確認可行的 key

    try:
        response = requests.post(API_URL, json=payload, timeout=10)
        if response.status_code == 200:
            data = response.json()
            # 根據你的 Log，回傳結構是 {'embeddings': [[...]]}
            if "embeddings" in data:
                vector = data["embeddings"][0]
                dim = len(vector)
                print(f"\n🎉 抓到了！")
                print(f"✅ 正確的 VECTOR_SIZE 應該設定為: {dim}")
                return dim
            else:
                print(f"⚠️ 格式怪怪的: {data.keys()}")
        else:
            print(f"❌ API 錯誤: {response.status_code}")
    except Exception as e:
        print(f"❌ 連線失敗: {e}")

check_dimension()